In [1]:
import ee

In [2]:
from agrigee_lite.ee_utils import ee_filter_img_collection_invalid_pixels

In [3]:
ee.Initialize(opt_url="https://earthengine-highvolume.googleapis.com", project="ee-mateuspsilva")

In [ ]:
def intersect_lists(list1, list2):
    return list1.map(lambda el: ee.Algorithms.If(list2.contains(el), el, None)).removeAll([None])

In [ ]:
# Define aoi e intervalo de datas
aoi = ee.Geometry.Polygon([
    [
        [-45.75383264123897, -11.695413119483494],
        [-45.74318963586788, -11.695413119483494],
        [-45.74318963586788, -11.6879326786798],
        [-45.75383264123897, -11.6879326786798],
        [-45.75383264123897, -11.695413119483494],
    ]
])
start_date = '2022-01-01'
end_date = '2022-12-31'
pixel_size = 10  # ou 30, dependendo do satélite

# Carrega coleções
l8_raw = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterBounds(aoi).filterDate(start_date, end_date)
s2_raw = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(aoi).filterDate(start_date, end_date)

# Aplica filtro de pixels válidos e adiciona ZZ_USER_TIME_DUMMY
l8 = ee_filter_img_collection_invalid_pixels(l8_raw, ee_geometry=aoi, pixel_size=30, min_valid_pixels=5)
s2 = ee_filter_img_collection_invalid_pixels(s2_raw, ee_geometry=aoi, pixel_size=10)

# Extrai datas únicas com base em ZZ_USER_TIME_DUMMY
def extract_dates(imgcol):
    return imgcol.aggregate_array('ZZ_USER_TIME_DUMMY').distinct()

l8_dates = extract_dates(l8)
s2_dates = extract_dates(s2)

common_dates = intersect_lists(l8_dates, s2_dates)

# Filtra coleções apenas para as datas em comum (comparando com ZZ_USER_TIME_DUMMY)
def filter_by_common_dates(ic, dates):
    return ic.filter(ee.Filter.inList('ZZ_USER_TIME_DUMMY', dates))

l8_filtered = filter_by_common_dates(l8, common_dates)
s2_filtered = filter_by_common_dates(s2, common_dates)


In [ ]:
l8_filtered

In [ ]:
s2_filtered

In [ ]:
l8_filtered

In [ ]:
s2_filtered

In [ ]:
import geemap

In [ ]:
l8_filtered

In [ ]:
# Define aoi e intervalo de datas
aoi = ee.Geometry.BBox(-50, -15, -49, -14)
start_date = '2022-01-01'
end_date = '2022-12-31'

# Coleções
l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate(start_date, end_date)

s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(aoi) \
    .filterDate(start_date, end_date)

# Extrai datas únicas (formato yyyy-mm-dd)
def extract_dates(ic, date_property='DATE'):
    return ic.map(lambda img: img.set(date_property, ee.Date(img.get('system:time_start')).format('yyyy-MM-dd'))) \
             .aggregate_array(date_property) \
             .distinct()

l8_dates = extract_dates(l8)
s2_dates = extract_dates(s2)

# Interseção de datas
common_dates = intersect_lists(l8_dates, s2_dates)

# Adiciona campo DATE (yyyy-MM-dd) para cada imagem
def add_date_string(img):
    return img.set('DATE', ee.Date(img.get('system:time_start')).format('YYYY-MM-dd'))

# Aplica isso nas duas coleções
l8 = l8.map(add_date_string)
s2 = s2.map(add_date_string)

def filter_by_dates(ic, dates):
    return ic.filter(ee.Filter.inList('DATE', dates))

l8_filtered = filter_by_dates(l8, common_dates)
s2_filtered = filter_by_dates(s2, common_dates)

In [ ]:
l8_filtered

In [ ]:
import geemap

In [ ]:
l8_filtered

In [ ]:
s2_filtered

In [ ]:
for x in l8_dates.getInfo():
    print(x, end=" ")

In [ ]:
for x in s2_dates.getInfo():
    print(x, end=" ")

In [ ]:
for x in common_dates.getInfo():
    print(x, end=" ")

In [ ]:
l8_filtered = filter_by_dates(l8, l8_dates)

In [ ]:
s2_filtered.size().getInfo()

In [ ]:
import geemap

In [ ]:
s2_filtered.getInfo()

In [ ]:
print()

In [ ]:
l8_filtered.size().getInfo()

In [ ]:
import agrigee_lite as agl

In [ ]:
s2_sat = agl.sat.Sentinel2()
l8_sat = agl.sat.Landsat8()

In [13]:
def extract_dates(imgcol):
    return imgcol.aggregate_array('ZZ_USER_TIME_DUMMY').distinct()

def filter_by_common_dates(ic, dates):
    return ic.filter(ee.Filter.inList('ZZ_USER_TIME_DUMMY', dates))

def intersect_lists(list1, list2):
    return list1.map(lambda el: ee.Algorithms.If(list2.contains(el), el, None)).removeAll([None])

def rename_bands(collection: ee.ImageCollection, prefix: str, postfix: str):
    def rename(image):
        image = ee.Image(image)
        band_names = image.bandNames()
        new_names = band_names.map(lambda name: ee.String(prefix).cat(name).cat(ee.String(postfix)))
        return image.select(band_names, new_names)
    return collection.map(rename)

In [4]:
from agrigee_lite.sat.abstract_satellite import OpticalSatellite

from functools import partial

from agrigee_lite.ee_utils import (
    ee_add_indexes_to_image,
    ee_cloud_probability_mask,
    ee_filter_img_collection_invalid_pixels,
    ee_get_number_of_pixels,
    ee_get_reducers,
    ee_map_bands_and_doy,
    ee_safe_remove_borders,
)

In [22]:
class UnifiedSatellite(OpticalSatellite):
    def __init__(self, satellite_a, satellite_b):
        super().__init__()
        self.sat_a = satellite_a
        self.sat_b = satellite_b

        self.startDate = satellite_a.startDate
        self.endDate = satellite_b.endDate
        self.pixelSize = min(satellite_a.pixelSize, satellite_b.pixelSize)

    def imageCollection(self, ee_feature: ee.Feature) -> ee.ImageCollection:
        sat_a = self.sat_a.imageCollection(ee_feature)
        sat_b = self.sat_b.imageCollection(ee_feature)

        sat_a_dates = extract_dates(sat_a)
        sat_b_dates = extract_dates(sat_b)

        common_dates = intersect_lists(sat_a_dates, sat_b_dates)

        sat_a_filtered = filter_by_common_dates(sat_a, common_dates)
        sat_b_filtered = filter_by_common_dates(sat_b, common_dates)

        sat_a_filtered = rename_bands(sat_a_filtered, "8", "sat_a")
        sat_b_filtered = rename_bands(sat_b_filtered, "7", "sat_b")

        merged = sat_a_filtered.linkCollection(sat_b_filtered, matchPropertyName="ZZ_USER_TIME_DUMMY", linkedBands=sat_b_filtered.first().bandNames())

        return merged

    def compute(
        self,
        ee_feature: ee.Feature,
        subsampling_max_pixels: float,
        reducers: list[str] | None = None,
    ) -> ee.FeatureCollection:
        ee_geometry = ee_feature.geometry()
        ee_geometry = ee_safe_remove_borders(ee_geometry, self.pixelSize, 35000)
        ee_feature = ee_feature.setGeometry(ee_geometry)

        s2_img = self.imageCollection(ee_feature)

        features = s2_img.map(
            partial(
                ee_map_bands_and_doy,
                ee_feature=ee_feature,
                pixel_size=self.pixelSize,
                subsampling_max_pixels=ee_get_number_of_pixels(ee_geometry, subsampling_max_pixels, self.pixelSize),
                reducer=ee_get_reducers(reducers),
            )
        )

        return features


In [6]:
import geopandas as gpd

In [10]:
import agrigee_lite as agl

In [7]:
gdf = gpd.read_parquet("data/sample.parquet")

In [8]:
row = gdf.iloc[3]

In [23]:
s2_sat = agl.sat.Sentinel2()
l8_sat = agl.sat.Landsat8()
merged_sat = UnifiedSatellite(s2_sat, l8_sat)

In [24]:
agl.get.sits(row.geometry, row.start_date, row.end_date, merged_sat)

,indexnum,timestamp,bluesat_b,greensat_b,redsat_b,nirsat_b,swir1sat_b,swir2sat_b,bluesat_a,greensat_a,redsat_a,re1sat_a,re2sat_a,re3sat_a,nirsat_a,re4sat_a,swir1sat_a,swir2sat_a,validPixelsCount
0,0,2020-12-17,0.035950,0.053742,0.031292,0.603083,0.227289,0.089713,0.032334,0.056738,0.030345,0.091900,0.450771,0.598475,0.577234,0.629825,0.266430,0.118397,48842
1,0,2021-03-07,0.059654,0.099062,0.127204,0.254190,0.325549,0.188960,0.115700,0.147400,0.175200,0.213200,0.251700,0.272700,0.270400,0.300700,0.389200,0.262200,38554
2,0,2021-05-26,0.025473,0.044736,0.027040,0.546154,0.183446,0.065595,0.029827,0.051100,0.029435,0.080226,0.416200,0.542706,0.534259,0.563078,0.205100,0.079200,49015
3,0,2021-08-14,0.070509,0.102001,0.160413,0.344087,0.412720,0.245085,0.073700,0.106821,0.180490,0.229582,0.264500,0.301636,0.326300,0.357551,0.455915,0.288263,49015


In [19]:
feature = ee.Feature(
    ee.Geometry.Polygon([
        [
            [-45.75383264123897, -11.695413119483494],
            [-45.74318963586788, -11.695413119483494],
            [-45.74318963586788, -11.6879326786798],
            [-45.75383264123897, -11.6879326786798],
            [-45.75383264123897, -11.695413119483494],
        ]
    ]),
    {"s": ee.Date('2022-01-01'), "e": ee.Date('2023-01-01'), "0": 0},
)

s2 = s2_sat.imageCollection(feature)
l8 = l8_sat.imageCollection(feature)

l8_dates = extract_dates(l8)
s2_dates = extract_dates(s2)

common_dates = intersect_lists(l8_dates, s2_dates)

l8_filtered = filter_by_common_dates(l8, common_dates)
s2_filtered = filter_by_common_dates(s2, common_dates)

l8_filtered = rename_bands(l8_filtered, "7", "l8")
s2_filtered = rename_bands(s2_filtered, "8", "s2")

In [20]:
merged = s2_filtered.linkCollection(l8_filtered, matchPropertyName="ZZ_USER_TIME_DUMMY", linkedBands=l8_filtered.first().bandNames())

In [ ]:
agl.ee_utils.ee_map_bands_and_doy?

In [21]:
features = merged.map(
    partial(
        agl.ee_utils.ee_map_bands_and_doy,
        ee_feature=feature,
        pixel_size=10,
        subsampling_max_pixels=100,
        reducer=agl.ee_utils.ee_get_reducers(["median"]),
    )
)

In [ ]:
features.getInfo()

In [ ]:
import geemap

In [ ]:
rename_bands(l8_filtered, "7", "l8")

In [ ]:
l8_filtered.size().getInfo()